## CML Wet/Dry classification with neural networks

### Steps:

1. Load data from NetCDF

This step is omitted since the full data set is not available (see README.md). However, there is some dummy code to illustrate how the data set is shaped.

2. Build CNN

This step is easy using the Keras API.

3. Train CNN

Also omitted since the training data is missing. The weights of the trained model are in the repository.

In [7]:
# imports
%matplotlib inline
import json
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.utils import shuffle
# neural network
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.optimizers import SGD
#import tensorflow for optional direct use
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [8]:
# fix random seed for reproducibility
random_state = 1234
np.random.seed(random_state)

The following cell is an example of how to use tensorflow to limit the memory that is used on the GPU

# Functions

In [9]:
###################
## Keras Metrics ##
###################

def matthews_correlation(y_true, y_pred):
    """
    Matthews correlation metric.
    It is only computed as a batch-wise average, not globally.
    Computes the Matthews correlation coefficient measure for quality
    of binary classifiers.
    """
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def tpr(y_true, y_pred):
    """
    True positive rate
    """
    y_pred_pos = K.round(y_pred)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(y_true)
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)/K.sum(y_pos)
    return tp

def tnr(y_true, y_pred):
    """
    True negative rate
    """
    y_pred_pos = K.round(y_pred)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(y_true)
    y_neg = 1 - y_pos
    tn = K.sum(y_neg * y_pred_neg)/K.sum(y_neg)
    return tn

def fpr(y_true, y_pred):
    """
    False positive rate
    """
    y_pred_pos = K.round(y_pred)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(y_true)
    y_neg = 1 - y_pos
    fp = K.sum(y_neg * y_pred_pos)/K.sum(y_neg)
    return fp

def fnr(y_true, y_pred):
    """
    False negative rate
    """
    y_pred_pos = K.round(y_pred)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(y_true)
    y_neg = 1 - y_pos
    fn = K.sum(y_pos * y_pred_neg)/K.sum(y_pos)
    return fn


# Load data

The shape of the input data (X_train and X_test) is (number of samples, window size in minutes (180), number of channels(2)). The shape of the binary reference data (y_train and y_test) is (number of samples).

# Convolutional Neural Network

In [10]:
# These are all parameters, that we have to specify for the model architecture.
window = 180

kernel_size = 3

dropout = 0.4

n_fc_neurons = 64

n_filters = [24, 48, 48, 96, 192]

batchsize = 10000

In [11]:
# Model architecture
# Convolutional part
input1 = Input(shape=(window,2,))
###
x1 =Conv1D(filters=n_filters[0], kernel_size=kernel_size, padding='same', activation='relu')(input1)
x1 = Conv1D(filters=n_filters[0], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = MaxPooling1D(pool_size=kernel_size)(x1)
###
x1 = Conv1D(filters=n_filters[1], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = Conv1D(filters=n_filters[1], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = MaxPooling1D(pool_size=kernel_size)(x1)
###
x1 = Conv1D(filters=n_filters[2], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = Conv1D(filters=n_filters[2], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = MaxPooling1D(pool_size=kernel_size)(x1)
###
x1 = Conv1D(filters=n_filters[3], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = Conv1D(filters=n_filters[3], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = MaxPooling1D(pool_size=kernel_size)(x1)
###
x1 = Conv1D(filters=n_filters[4], kernel_size=kernel_size, padding='same', activation='relu')(x1)
x1 = GlobalAveragePooling1D()(x1)
# FC part
x1 = Dense(n_fc_neurons, activation='relu')(x1)
x1 = Dropout(dropout)(x1)
x1 = Dense(n_fc_neurons, activation='relu')(x1)
x1 = Dropout(dropout)(x1)
out = Dense(1, activation='sigmoid')(x1)

model = keras.models.Model(inputs=input1, outputs=out)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 180, 2)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 180, 24)           168       
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 180, 24)           1752      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 60, 24)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 60, 48)            3504      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 60, 48)            6960      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 20, 48)            0         
__________

In [12]:
# The model needs to be compiled first
model.compile(loss='binary_crossentropy',
             optimizer=SGD(lr=0.008, decay =1e-3, momentum=0.9, nesterov=True),
             metrics=['accuracy', 'mse', matthews_correlation , tpr, tnr, fpr, fnr]
             )
# optional callbacks for the training
callbacks_list = []

The number of training samples is reduced to a multiple of the batch size

## Training

## Save model, weights and training history

### serialize model to JSON

### serialize weights to HDF5

### save training history